In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# import required packages
import pandas as pd
import numpy as np

# preprocessing
from sklearn.model_selection import train_test_split

# models
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV 

# metrics
import time
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn.metrics import recall_score, roc_auc_score

# plots
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

In [11]:
# load the cleaned and preprocessed data sets
df_train = pd.read_csv('../data/train.csv')
df_valid = pd.read_csv('../data/valid.csv')
df_test = pd.read_csv('../data/test.csv')

# include validation set into train set
df_train = pd.concat([df_train, df_valid]).sample(frac=1, random_state=42).reset_index(drop=True)

# shape of the train and test sets: (rows, columns)
display(df_train.shape, df_test.shape)

(69767, 78)

(17442, 78)

In [12]:
# first 5 datapoints of train, validation and test sets
display(df_train.head())
display(df_test.head())

,qty_dot_url,qty_hyphen_url,qty_underline_url,qty_slash_url,qty_equal_url,qty_at_url,qty_and_url,qty_comma_url,qty_plus_url,qty_asterisk_url,...,time_domain_activation,time_domain_expiration,qty_ip_resolved,qty_nameservers,qty_mx_servers,ttl_hostname,tls_ssl_certificate,qty_redirects,url_shortened,phishing
0,1,0,1,3,0,0,0,0,0,0,...,7404,264,1,4,2,1411,1,1,0,0
1,3,1,0,4,0,0,0,0,0,0,...,-1,-1,1,2,1,281,0,0,0,1
2,2,0,0,0,0,0,0,0,0,0,...,7372,662,1,2,1,14398,1,0,0,0
3,14,1,0,5,3,0,8,0,0,0,...,235,129,1,2,1,7764,1,0,0,1
4,2,0,0,1,0,0,0,0,0,0,...,648,81,1,2,1,14391,1,0,0,1


,qty_dot_url,qty_hyphen_url,qty_underline_url,qty_slash_url,qty_equal_url,qty_at_url,qty_and_url,qty_comma_url,qty_plus_url,qty_asterisk_url,...,time_domain_activation,time_domain_expiration,qty_ip_resolved,qty_nameservers,qty_mx_servers,ttl_hostname,tls_ssl_certificate,qty_redirects,url_shortened,phishing
0,3,0,0,2,1,0,0,0,0,0,...,6007,201,1,3,1,288,0,0,0,0
1,2,0,1,2,0,0,0,0,0,0,...,1126,334,2,2,0,591,0,5,0,1
2,3,0,0,1,0,0,0,0,0,0,...,5820,626,1,2,1,14393,1,1,0,1
3,2,1,0,3,0,0,0,0,0,0,...,3066,220,1,2,2,594,0,0,0,1
4,2,0,2,1,0,0,0,0,0,0,...,4023,359,3,2,0,298,0,0,0,1
